1. Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [204]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA

In [205]:
X, y = load_iris(return_X_y=True)

In [206]:
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [207]:
# отмасштабируем выборку
X = X.astype(float)
X = standard_scale(X)

Я использовал модифицированный метод KNN с учетом весов

In [208]:
def knn_weighted(x_train, y_train, x_test, k):
    answers = []
    for x in x_test:

        distance = np.linalg.norm(x_train - x, ord=2, axis=1)

        test_distances = zip(distance, y_train)

        classes = {class_item: 0 for class_item in set(y_train)}

        for d in sorted(test_distances)[:k]:
            classes[d[1]] += (0.5 ** d[0])

        answers.append(sorted(classes, key=classes.get)[-1])

    return answers

In [209]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [210]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape

((112, 4), (38, 4))

Посчитаем на полной выборке

In [271]:
k = 10

y_pred = knn_weighted(X_train, y_train, X_test, k)

print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 10: 0.974


In [219]:
# оформим алгоритм в функцию и добавим возможность выбора количества компонент
def pca_man(x, components=1):

    covariance_matrix = x.T @ x

    eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

    # сформируем список кортежей (собственное значение, собственный вектор)
    eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

    # и отсортируем список по убыванию собственных значений
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    # проверка на количество передаваемых компонент
    if components > x.shape[1]:
        raise ValueError("Amount of principal components cannot exceed amount of features")

    W = np.hstack([eig_pairs[i][1].reshape((len(eig_pairs[0][1])),1) for i in range(components)])

    Z = X.dot(W)

    return Z

In [220]:
x_decreased = pca_man(X, 2)

In [221]:
X_train, X_test, y_train, y_test = train_test_split(x_decreased, y,
                                                    test_size=0.25,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape

((112, 2), (38, 2))

In [222]:
k = 10

y_pred = knn_weighted(X_train, y_train, X_test, k)

print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 10: 0.868
Wall time: 3.01 ms


Точность незначительно, но упала. Это обусловлено влиянием отброшенных компонент. Для сравнения снизим размерность данных, используя три главных компоненты.

In [223]:
x_decreased = pca_man(X, 3)

In [224]:
X_train, X_test, y_train, y_test = train_test_split(x_decreased, y,
                                                    test_size=0.25,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape

((112, 3), (38, 3))

In [272]:
k = 10

y_pred = knn_weighted(X_train, y_train, X_test, k)

print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 10: 0.974


Как видно, точность выросла до уровня полной выборки.

In [269]:
def get_info_share(x):
    covariance_matrix = x.T @ x

    eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

    # сформируем список кортежей (собственное значение, собственный вектор)
    eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

    # и отсортируем список по убыванию собственных значений
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    eig_sum = sum(eig_values)
    var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
    cum_var_exp = np.cumsum(var_exp)
    print(f'Количество компонент: {X.shape[1]}\n')
    print(f'Доля дисперсии, описываемая каждой из компонент:')
    for i, j in enumerate(var_exp,1):
        print(f'Компонента {i}: {j}')

    # а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
    print(f'\nКумулятивная доля дисперсии по компонентам')
    for i, j in enumerate(cum_var_exp,1):
        print(f'Компонента {i}: {j:.4f}%')


In [270]:
get_info_share(X)

Количество компонент: 4

Доля дисперсии, описываемая каждой из компонент:
Компонента 1: 72.96244541329987
Компонента 2: 22.850761786701774
Компонента 3: 3.6689218892828697
Компонента 4: 0.517870910715493

Кумулятивная доля дисперсии по компонентам
Компонента 1: 72.9624%
Компонента 2: 95.8132%
Компонента 3: 99.4821%
Компонента 4: 100.0000%


Таким образом влияние третьей компоненты примерно 3,67% от общей информации, что достаточно чтобы снизить точность с 0,974 до 0,868